In [ ]:
import bert_score as bert
bert.__version__
import csv
import torch

import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

import gc

torch.cuda.empty_cache()
gc.collect()

In [ ]:
model_name = "microsoft/mdeberta-v3-base"
num_layers = 10

#We use the scorer, so we don't have to load the model every time
mdeberta_scorer = bert.BERTScorer(model_type=model_name, lang="et", num_layers=num_layers, rescale_with_baseline=False)

In [ ]:
def ReadTxtFileAsOneString(path):
    with open(f"{path}", "r", encoding="utf-8") as file:
        txt = file.read().strip().replace("\n", " ")    
    return txt

## Compare AI summary against doctor written summary

In [ ]:
print("Comparing")
letters = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]

csv_filename = "bertscore_against_doctor_summary_f1_score_10.csv"

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file, delimiter="\t")
    writer.writerow(["Doctor", "Patient", 
                        "Prompt 1", "Prompt 1", "Prompt 1", "Prompt 1", "Prompt 1", "Prompt 1", "Prompt 1", "Prompt 1", "Prompt 1", "Prompt 1",  
                        "Prompt 2", "Prompt 2", "Prompt 2", "Prompt 2", "Prompt 2", "Prompt 2", "Prompt 2", "Prompt 2", "Prompt 2", "Prompt 2",   
                        "Prompt 3", "Prompt 3", "Prompt 3", "Prompt 3", "Prompt 3", "Prompt 3", "Prompt 3", "Prompt 3", "Prompt 3", "Prompt 3",  
                        "Prompt 4", "Prompt 4", "Prompt 4", "Prompt 4", "Prompt 4", "Prompt 4", "Prompt 4", "Prompt 4", "Prompt 4", "Prompt 4",  
                        "Prompt 5", "Prompt 5", "Prompt 5", "Prompt 5", "Prompt 5", "Prompt 5", "Prompt 5", "Prompt 5", "Prompt 5", "Prompt 5"])
    writer.writerow(["Doctor", "Patient", 
                        "a", "b", "c", "d", "e", "f", "g", "h", "i", "j",
                        "a", "b", "c", "d", "e", "f", "g", "h", "i", "j",
                        "a", "b", "c", "d", "e", "f", "g", "h", "i", "j",
                        "a", "b", "c", "d", "e", "f", "g", "h", "i", "j",
                        "a", "b", "c", "d", "e", "f", "g", "h", "i", "j"])

for doctor_index in range(1,11):
    for patient_index in range(1,11):
        print("New row")
        row = list()
        row.append(doctor_index)
        row.append(patient_index)
        doctor_summary = ReadTxtFileAsOneString(f"arst_{doctor_index:03}/patsient_{patient_index:03}/toorfailid/doctor_indexi_kokkuvote_orig_{doctor_index:02}_{patient_index:02}.txt")
        for prompt in range(1,6):
            for letter in letters:
                generated_summary = ReadTxtFileAsOneString(f"summaries/arst_{doctor_index}_patsient_{patient_index}/kokkuvõtted/prompt_{prompt}/arst_{doctor_index:02}_patsient_{patient_index:02}_kokkuvõte_prompt_{prompt}_{letter}.txt")

                print(f"\nComparing doctor: {doctor_index}, patient: {patient_index}, prompt: {prompt}, letter: {letter}.")
                P, R, F1 = mdeberta_scorer.score([doctor_summary], [generated_summary])
                #print(f"P tensors: {P}")
                #print(f"R tensors: {R}")
                #print(f"F1 tensors: {F1}")

                # We are mostly interested in the F1-score
                f1_score = F1.item()
                print(f"F1 score: {f1_score:.4f}")
                row.append(round(f1_score, 3))

        with open(csv_filename, mode="a", newline="") as file:
            writer = csv.writer(file, delimiter="\t")  # Use tab separator
            writer.writerow(row)




In [ ]:
import os
print("Comparing")


csv_filename = "bertscore_against_doctor_summary_f1_score_clean.csv"


with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file, delimiter="\t")
    writer.writerow(["Doctor", "Patient", "Prompt 1", "Prompt 2", "Prompt 3", "Prompt 4", "Prompt 5"])


for doctor_index in range(5,7):
    for patient_index in range(10,11):
        print("New row")
        row = list()
        row.append(doctor_index)
        row.append(patient_index)
        doctor_summary = ReadTxtFileAsOneString(f"Arst_{doctor_index:03}/Patsient_{patient_index:03}/toorfailid/arsti_kokkuvote_orig_{doctor_index:02}_{patient_index:02}.txt")
        for prompt in range(1,6):
            letter = "puhas"
            generated_summary_path = f"summaries/arst_{doctor_index}_patsient_{patient_index}/kokkuvõtted/prompt_{prompt}/arst_{doctor_index:02}_patsient_{patient_index:02}_kokkuvõte_prompt_{prompt}_{letter}.txt"
            
            if os.path.exists(generated_summary_path):
                generated_summary = ReadTxtFileAsOneString(generated_summary_path)
                print(f"\nComparing doctor_index {doctor_index}, patient_index {patient_index}, prompt {prompt}, letter {letter}")
                P, R, F1 = mdeberta_scorer.score([doctor_summary], [generated_summary])
                #print(f"P tensors: {P}")
                #print(f"R tensors: {R}")
                #print(f"F1 tensors: {F1}")

                f1_score = F1.item()
                print(f"F1 score: {f1_score:.4f}")
                row.append(round(f1_score, 3))
            else:
                print("File not found. Appending 0.")
                row.append(0)

        with open(csv_filename, mode="a", newline="") as file:
            writer = csv.writer(file, delimiter="\t")  # Use tab separator
            writer.writerow(row)